In [1]:
from raid.utils import load_data

train_df = load_data(split="train",include_adversarial=False)

Human model will have NaN decoding duh!

In [2]:
train_df['decoding'].unique()

array([nan, 'greedy', 'sampling'], dtype=object)

In [3]:
train_df['domain'].value_counts()

domain
books        62335
news         62300
reddit       62265
wiki         62265
recipes      62020
poetry       61985
abstracts    61810
reviews      33005
Name: count, dtype: int64

In [4]:
train_df[train_df["model"] == "human"][
    "domain"
].value_counts()

domain
books        1781
news         1780
reddit       1779
wiki         1779
recipes      1772
poetry       1771
abstracts    1766
reviews       943
Name: count, dtype: int64

In [2]:
import numpy as np
train_df['model'] = np.where(train_df['model'] == 'human',0,1)

In [3]:
train_df['model'].value_counts()

model
1    454614
0     13371
Name: count, dtype: int64

The class imbalance for Ai vs Human is 35:1. So to combat this we have to use class weighting. Adding this weight = (total number of samples)/(number of classes * number of samples)

In [4]:
train_df.head()

,id,adv_source_id,source_id,model,decoding,repetition_penalty,attack,domain,title,prompt,generation
0,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,e5e058ce-be2b-459d-af36-32532aaba5ff,0,NaN,NaN,none,abstracts,FUTURE-AI: Guiding Principles and Consensus Re...,NaN,The recent advancements in artificial intellig...
1,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,f95b107b-d176-4af5-90f7-4d0bb20caf93,0,NaN,NaN,none,abstracts,EdgeFlow: Achieving Practical Interactive Segm...,NaN,High-quality training data play a key role in ...
2,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,856d8972-9e3d-4544-babc-0fe16f21e04d,0,NaN,NaN,none,abstracts,Semi-supervised Contrastive Learning for Label...,NaN,The success of deep learning methods in medica...
3,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,fbc8a5ea-90fa-47b8-8fa7-73dd954f1524,0,NaN,NaN,none,abstracts,Combo Loss: Handling Input and Output Imbalanc...,NaN,Simultaneous segmentation of multiple organs f...
4,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,72c41b8d-0069-4886-b734-a4000ffca286,0,NaN,NaN,none,abstracts,Attention-Based 3D Seismic Fault Segmentation ...,NaN,Detection faults in seismic data is a crucial ...


In [5]:
train_df = train_df.drop(
    [
        "id",
        "decoding",
        "domain",
        "adv_source_id",
        "source_id",
        "repetition_penalty",
        "attack",
        "title",
        "prompt"
    ],
    axis=1,
)

train_df = train_df.rename(columns={
    "generation" : "content",
    "model" : "label"
})

train_df.head()

,label,content
0,0,The recent advancements in artificial intellig...
1,0,High-quality training data play a key role in ...
2,0,The success of deep learning methods in medica...
3,0,Simultaneous segmentation of multiple organs f...
4,0,Detection faults in seismic data is a crucial ...


In [6]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

labels = train_df["label"]

class_weights = compute_class_weight("balanced",classes=np.unique(labels),y=labels)

print(class_weights)

[17.5         0.51470588]


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
import torch
import torch.nn as nn

weights = torch.tensor(class_weights, dtype=torch.float)

model_path = "distilbert/distilroberta-base"

loss_fn = nn.CrossEntropyLoss(weight=weights)

model = AutoModelForSequenceClassification.from_pretrained(model_path,num_labels=2)
tokeinzer = AutoTokenizer.from_pretrained(model_path)   

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilbert/distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df["label"])
train_dataset = Dataset.from_pandas(train_data)

val_data,test_data = train_test_split(val_data, test_size=0.5, random_state=42, stratify=val_data["label"])
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

In [11]:
train_dataset

Dataset({
    features: ['label', 'content'],
    num_rows: 421186
})

In [10]:
train_dataset = train_dataset.remove_columns(["__index_level_0__"])
val_dataset = val_dataset.remove_columns(["__index_level_0__"])
test_dataset = test_dataset.remove_columns(["__index_level_0__"])

In [14]:
def preprocess_fn(batch):
    return tokeinzer(batch["content"], truncation=True, padding="max_length", max_length=512)

train_dataset = train_dataset.map(preprocess_fn, batched=True)
val_dataset = val_dataset.map(preprocess_fn, batched=True)
test_dataset = test_dataset.map(preprocess_fn, batched=True)

Map:   0%|          | 0/421186 [00:00<?, ? examples/s]

Map:   0%|          | 0/23399 [00:00<?, ? examples/s]

Map:   0%|          | 0/23400 [00:00<?, ? examples/s]

In [15]:
from transformers import TrainingArguments, Trainer


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss
    
training_args = TrainingArguments(
    output_dir="./checkpoints",
    eval_strategy="epoch",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=3e-5,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    run_name="distilroberta-noadv-raid"
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokeinzer
)

/tmp/ipykernel_14465/165361310.py:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(


In [16]:
trainer.train()

TypeError: CustomTrainer.compute_loss() got an unexpected keyword argument 'num_items_in_batch'